In [3]:

import numpy as np
import cupy as cp
import h5py
import os
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

In [4]:
#选择要测试kpi sms=0 call=1 internet=2
kpi=0
if kpi==0:
    kpi_name='sms'
elif kpi==1:
    kpi_name='call'
else:
    kpi_name='internet'

In [5]:
f = h5py.File('D:/library/Github/Innovative-practice/library/csv/第三学期/trainX400conv.h5', 'r')
trainX = f['data']
trainX  = cp.array(trainX)
trainX = trainX[:,:,:,:,kpi]
f.close()
f = h5py.File('D:/library/Github/Innovative-practice/library/csv/第三学期/trainY400conv.h5', 'r')
trainY = f['data']
trainY  = cp.array(trainY)
trainY = trainY[:,:,:,kpi]
f.close()
trainX=cp.reshape(trainX,(trainX.shape[0], trainX.shape[1], trainX.shape[2],trainX.shape[3],1))
trainY=cp.reshape(trainY,(trainY.shape[0], trainY.shape[1], trainY.shape[2],1))
print(trainX.shape)
print(trainY.shape)

(1017, 24, 20, 20, 1)
(1017, 20, 20, 1)


In [11]:
trainmax=int(cp.max(trainX[:,:,:,:,:]))
testmax=int(cp.max(trainY[:,:,:,:]))
print(trainmax)
print(testmax)

4047
4047


In [12]:
trainX = trainX/trainmax
trainY = trainY/trainmax

In [13]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None, *trainX.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=False,
    activation="relu",
)(x)
x = layers.Conv2D(
    filters=1, kernel_size=(3, 3), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.summary()
model.compile(
    loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 20, 20, 1)  0         
                             ]                                   
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, None, 20, 20, 64)  416256    
                                                                 
 batch_normalization (BatchN  (None, None, 20, 20, 64)  256      
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, None, 20, 20, 64)  295168    
                                                                 
 batch_normalization_1 (Batc  (None, None, 20, 20, 64)  256      
 hNormalization)                                                 
                                                             

In [ ]:
from tensorflow.python import test


# Define modifiable training hyperparameters.
epochs = 20
batch_size = 2
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit the model to the training data.
model.fit(
    trainX.get(),
    trainY.get(),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[tensorboard_callback],
    
)
model.save(os.path.join("D:\library\Github\Innovative-practice\第三学期\DATA",f"Test_{kpi_name}400" + ".h5"))